# First MSSP-PD(ABP) example

In [14]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
from collections import namedtuple

## Manage datas

Read the synthetic instance of Section 3.2

In [15]:
nodes_number = 25
agents_number = 4
synthetic_5x5_df = pd.read_csv("data/d_it_ij_5x5_1it.csv",
                               header=0,
                               names=["i", "j", "d_ij"])

# convert d_ij column into float
# print(synthetic_5x5_df.dtypes)
synthetic_5x5_df["d_ij"] = synthetic_5x5_df["d_ij"].str.replace(
    ",", ".").astype(np.float64)
# print(synthetic_5x5_df.dtypes)

# counting network nodes starting from 0
synthetic_5x5_df[["i", "j"]] = synthetic_5x5_df[["i", "j"]] - 1


Set up datas that will be used to solve the problem

In [16]:
Arc = namedtuple("Arc", synthetic_5x5_df.columns)
arcs = [Arc(*row) for row in synthetic_5x5_df.itertuples(index=False)]  # as List to avoid nusty bugs

nodes = np.unique(np.concatenate((synthetic_5x5_df["i"].to_numpy(),
                              synthetic_5x5_df["j"].to_numpy())))
agent_sources = np.array([0, 2, 3, 4])
agent_terminus = np.array([20, 22, 23, 24])

agents = range(agents_number)
# D = synthetic_5x5_df["d_ij"].to_numpy()

## Manage the problem

Create the problem

In [17]:
MSPP_PD_ABP_pb = gb.Model("First MSPP_PD_ABP_pb")
MSPP_PD_ABP_pb.setParam("OutputFlag", 0)

Define decision variables

In [18]:
X_var_shape = len(nodes), len(nodes), agents_number
Psi_var_shape = len(nodes), len(nodes)

X = MSPP_PD_ABP_pb.addMVar(X_var_shape,
                           vtype=GRB.BINARY,  # 5) Binary constraints
                           name="k-th agent traverse arc (i,j)")

Psi = MSPP_PD_ABP_pb.addMVar(Psi_var_shape,
                             vtype=GRB.BINARY,  # 8) Binary constraints
                             name="more agents traverse arc (i,j)")


Define the objective function

In [19]:
# 1-3) Objective function
distance_objn = gb.quicksum(
    arc.d_ij*X[arc.i, arc.j, k]
        for arc in arcs for k in agents
)
penalties_objn = gb.quicksum(
    Psi[arc.i, arc.j] for arc in arcs
)


MSPP_PD_ABP_pb.setObjectiveN(distance_objn, index=0, weight=0.5,
name="Distance")
MSPP_PD_ABP_pb.setObjectiveN(penalties_objn, index=1, weight=0.5,
name="Penalty")


MSPP_PD_ABP_pb.ModelSense = GRB.MINIMIZE

Add constraints

In [20]:
# 4) Flow constraints

def compute_flow(X, node, arcs, agent):
    flow_out = gb.quicksum(X[node, arc.j, agent] for arc in arcs if arc.i==node)
    flow_in = gb.quicksum(X[arc.i, node, agent] for arc in arcs if arc.j==node)
    return flow_out - flow_in


for k in agents:
    for i in nodes:
        if i == agent_sources[k]:
            MSPP_PD_ABP_pb.addConstr(compute_flow(X, i, arcs, k) == 1,
            name=f"Flow constr related to agent {k} in node {i}")
        elif i == agent_terminus[k]:
            MSPP_PD_ABP_pb.addConstr(compute_flow(X, i, arcs, k) == -1,
            name=f"Flow constr related to agent {k} in node {i}")
        else:
            MSPP_PD_ABP_pb.addConstr( compute_flow(X, i, arcs, k) == 0,
            name=f"Flow constr related to agent {k} in node {i}")
            

In [21]:
# 7) Binary penalization constraints

for arc in arcs:
    MSPP_PD_ABP_pb.addConstr(
        1/len(agents) * (gb.quicksum(X[arc.i, arc.j, k] for k in agents) - 1)
        <= Psi[arc.i, arc.j]
    ) 

Solve the problem

In [22]:
MSPP_PD_ABP_pb.optimize()

Report results

In [23]:
print("Result of the optimization is:")
if MSPP_PD_ABP_pb.Status == 2:
    print("optimal")
elif MSPP_PD_ABP_pb.Status == 3:
    print("infeasible")
elif MSPP_PD_ABP_pb.Status == 5:
    print("unbounded")
else:
    print("Some other return status")

Result of the optimization is:
optimal


In [24]:
n_objectives = MSPP_PD_ABP_pb.NumObj
n_solutions = MSPP_PD_ABP_pb.SolCount

print (f"The optimization founds {n_solutions} solutions:")
for sol in range(n_solutions):
    MSPP_PD_ABP_pb.params.SolutionNumber = sol

    print(f"Solution {sol}:", end="")
    for obj in range(n_objectives):
        MSPP_PD_ABP_pb.params.ObjNumber = obj
        print(f" {MSPP_PD_ABP_pb.ObjNName}={MSPP_PD_ABP_pb.ObjNVal}", end="")

    print()

    for k in agents:
        print(f"Agent {k} will follow the path:")
        for arc in arcs:
            if math.isclose(X.X[arc.i, arc.j, k], 1):
                print(f"{arc.i}->{arc.j}", end="\t")
        print("\n")

    print()

#! In which sense Solution 2 is a pareto-opt solution??
#TODO: UNDERSTAND IT!!

The optimization founds 3 solutions:
Solution 0: Distance=16.0 Penalty=1.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	

Agent 1 will follow the path:
2->7	7->12	12->18	18->22	

Agent 2 will follow the path:
3->7	7->12	12->17	17->23	

Agent 3 will follow the path:
4->8	8->14	14->18	18->24	


Solution 1: Distance=18.6 Penalty=0.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	

Agent 1 will follow the path:
2->7	7->12	12->18	18->22	

Agent 2 will follow the path:
3->7	7->12	12->17	17->23	

Agent 3 will follow the path:
4->8	8->14	14->18	18->24	


Solution 2: Distance=24.0 Penalty=0.0
Agent 0 will follow the path:
0->6	6->12	12->16	16->20	

Agent 1 will follow the path:
2->7	7->12	12->18	18->22	

Agent 2 will follow the path:
3->7	7->12	12->17	17->23	

Agent 3 will follow the path:
4->8	8->14	14->18	18->24	


